In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation , AveragePooling2D , Input ,Dropout
from tensorflow.keras.layers import Dense,  MaxPooling2D, Add, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow_addons as tfa

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3548)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

# 테스트 파일을  generator로 예측하신다면 아래 코드 !

In [2]:
# test 데이터 안 '6' 폴더에 있는 체크포인트 오류 데이터 제거 후 실행
path = './public'
test_dirs = path + '/test'
files = []
ids=[]
for img_cat in os.listdir(test_dirs):
    id_dir =  img_cat
    for filename in os.listdir(test_dirs + '/' + id_dir):
        files.append(id_dir + '/' +filename)
        ids.append(filename.split('.JPG')[0])
                           
test_data = pd.DataFrame(
                    {"file":files,
                    "id":ids}
                )    

display(test_data.head())
print(test_data.shape)

,file,id
0,0/0hmnf5orki.JPG,0hmnf5orki
1,0/0bgj9co0zl.JPG,0bgj9co0zl
2,0/03123sl42g.JPG,03123sl42g
3,0/0vwaki2su2.JPG,0vwaki2su2
4,0/09jgq862fk.JPG,09jgq862fk


(37964, 2)


In [3]:
# 데이터 형상 관련 상수 정의 (훈련할때 썼던 이미지 크기 사용)
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNEL=3
batch_size=128

In [4]:
# test
with tf.device('/device:GPU:0'):
    test_datagen=ImageDataGenerator(rescale=1./255) # MinMaxScaling
    test_generator=test_datagen.flow_from_dataframe(
        test_data,
        test_dirs,
        x_col= "file",
        y_col= "id",
        target_size = IMAGE_SIZE,
        class_mode = "raw",
        batch_size = batch_size,
        shuffle=False)


Found 37964 validated image filenames.


In [5]:
# 모델불러오기
with tf.device('/device:GPU:0'):

    model = load_model('./landmark_MobileNetv2_model_270_480_radam_2.h5', compile=False)

In [ ]:
with tf.device('/device:GPU:0'):
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.RectifiedAdam(),
                metrics=['accuracy'])
    pred = model.predict(test_generator)

In [7]:
pred_df = pd.DataFrame({'id':test_data['id'].values,
                       'landmark_id':np.argmax(pred, axis=1),
                       'conf':np.max(pred, axis=1)})

In [10]:
submission = pd.read_csv(path + '/sample_submisstion.csv')
sub = pd.merge(submission[['id']], pred_df, on='id', how='left')
sub.head()


,id,landmark_id,conf
0,xlf1tgh2ih,956,1.000000
1,68a3ot4osk,956,0.999990
2,si2lek4u0a,956,0.999785
3,rmtqxhipnv,956,0.936642
4,2flmjdud0e,956,1.000000


In [11]:
sub.to_csv(path + '/Densenet.csv', encoding='cp949', index=False)

# 테스트 파일을  TFRecord로 예측하신다면 아래 코드 ! 

In [ ]:
with tf.device('/device:GPU:1'):
    path = './public'
    test_tfrecord_path = path + '/tf_record_test.tfrecords'
    BATCH_SIZE = 128
    NUM_CLASS = 1049
    img_size = (270,480) # <- 학습할때 썼던 이미지 사이즈 입력해주세요 !
    
    image_feature_description_test = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'id': tf.io.FixedLenFeature([], tf.string),
    
    }


    def _parse_image_function_test(example_proto):
        return tf.io.parse_single_example(example_proto, image_feature_description_test)

    def map_func_test(target_record):
        img = target_record['image_raw']
        label = target_record['id']
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.dtypes.cast(img, tf.float32)
        return img, label

    def prep_func_test(image, label):
        result_image = image / 255
        result_image = tf.image.resize(result_image, img_size)

        return result_image, label

    test_dataset = tf.data.TFRecordDataset(test_tfrecord_path, compression_type='GZIP')
    test_dataset = test_dataset.map(_parse_image_function_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.map(map_func_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.map(prep_func_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    
    # 모델불러오기
    model = load_model('./landmark_MobileNetv2_model_270_480_radam_2.h5', compile=False) # 학습하셨던 모델을 불러와주세요
    id_list = []
    for _, label in test_dataset:
        id_list.extend(list(label))

    pred = model.predict(test_dataset)
    
    pred_df = pd.DataFrame({'id':id_list,
                       'landmark_id':np.argmax(pred,axis=1),
                       'conf':np.max(pred,axis=1)})
    pred_df['id'] = pred_df['id'].apply(lambda x : x.numpy().decode('utf-8'))
    
    submission = pd.read_csv(path + '/sample_submisstion.csv')
    sub = pd.merge(submission[['id']], pred_df, on='id', how='left')
    sub.head()

In [ ]:
sub.to_csv(path + '/Densenet.csv', encoding='cp949', index=False) # 저장할 파일 명 정해서 제출파일 만들어주세요~~~